![reduce](https://1.bp.blogspot.com/-6XdrSaHdjiw/XaaY2AUIxoI/AAAAAAAAGS4/MfBcvYjBVT0F9OJAkD4V9A7kD2FfFY7mACLcBGAsYHQ/s1600/reducing1.jpg)

# Reducing Memory Size for Great Energy Predictor III Competition

# Objective of the Kernel: Save Time & Memory

**If you would like to create a kernel for this Competition. This is a good idea to add this kernel as a data set to your own kernel. due to you can save your time and memory.**

**MEMORY USAGE BEFORE AND AFTER COMPLETION FOR TRAIN:**

Memory usage before running this script :** 2.6+ GB**

Memory usage after running this script :  **1.1+ GB**

This is **~ 60 %** of the initial size

**MEMORY USAGE BEFORE AND AFTER COMPLETION FOR TEST:**

Memory usage before running this script : **5.3+ GB**

Memory usage after running this script: **2.3+ GB**

This is **~ 60 %** of the initial size

# Impot

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import gc
gc.enable()
import time
import warnings
warnings.filterwarnings("ignore")

**What do we have in input**

In [ ]:
#print(os.listdir("../input"))
!ls -GFlash  ../input/ashrae-energy-prediction

# Import Dataset and creat our train & test dataset

In [ ]:
%%time
# import Dataset to play with it
train_data = pd.read_csv('/kaggle/input/ashrae-energy-prediction/train.csv')
building = pd.read_csv('/kaggle/input/ashrae-energy-prediction/building_metadata.csv')
weather_train = pd.read_csv('/kaggle/input/ashrae-energy-prediction/weather_train.csv')
train_data = train_data.merge(building, on='building_id', how='left')
train_data = train_data.merge(weather_train, on=['site_id', 'timestamp'], how='left')

test_data = pd.read_csv('/kaggle/input/ashrae-energy-prediction/test.csv')
weather_test = pd.read_csv('/kaggle/input/ashrae-energy-prediction/weather_test.csv')
test_data = test_data.merge(building, on='building_id', how='left')
test_data = test_data.merge(weather_test, on=['site_id', 'timestamp'], how='left')

print ("Done!")

In [ ]:
print('Shape of Data:')
print(train_data.shape)
print(test_data.shape)

# Before Reducing Memory

**When I have just read the data set and join them! I saw that the status of my RAM is more than 10GB!**
![ram1](https://1.bp.blogspot.com/-odGlxafFW2s/XaaZOgQUjQI/AAAAAAAAGTA/o6KnYfo4Ya4G0wnaWo0AuclzRQEP3EhSwCLcBGAsYHQ/s1600/ram1.PNG)

**Then we shoud just delete some dt!**

In [ ]:
del building, weather_train, weather_test

**Now let's check the size of our train & test**

In [ ]:
train_data.info()

In [ ]:
test_data.info()

# Reducing Memory Size

**It is necessary that after using this code, carefully check the output results for each column.**

In [ ]:
#Based on this great kernel https://www.kaggle.com/arjanso/reducing-dataframe-memory-size-by-65
def reduce_mem_usage(df):
    start_mem_usg = df.memory_usage().sum() / 1024**2 
    print("Memory usage of properties dataframe is :",start_mem_usg," MB")
    NAlist = [] # Keeps track of columns that have missing values filled in. 
    for col in df.columns:
        if df[col].dtype != object:  # Exclude strings            
            # Print current column type
            print("******************************")
            print("Column: ",col)
            print("dtype before: ",df[col].dtype)            
            # make variables for Int, max and min
            IsInt = False
            mx = df[col].max()
            mn = df[col].min()
            print("min for this col: ",mn)
            print("max for this col: ",mx)
            # Integer does not support NA, therefore, NA needs to be filled
            if not np.isfinite(df[col]).all(): 
                NAlist.append(col)
                df[col].fillna(mn-1,inplace=True)  
                   
            # test if column can be converted to an integer
            asint = df[col].fillna(0).astype(np.int64)
            result = (df[col] - asint)
            result = result.sum()
            if result > -0.01 and result < 0.01:
                IsInt = True            
            # Make Integer/unsigned Integer datatypes
            if IsInt:
                if mn >= 0:
                    if mx < 255:
                        df[col] = df[col].astype(np.uint8)
                    elif mx < 65535:
                        df[col] = df[col].astype(np.uint16)
                    elif mx < 4294967295:
                        df[col] = df[col].astype(np.uint32)
                    else:
                        df[col] = df[col].astype(np.uint64)
                else:
                    if mn > np.iinfo(np.int8).min and mx < np.iinfo(np.int8).max:
                        df[col] = df[col].astype(np.int8)
                    elif mn > np.iinfo(np.int16).min and mx < np.iinfo(np.int16).max:
                        df[col] = df[col].astype(np.int16)
                    elif mn > np.iinfo(np.int32).min and mx < np.iinfo(np.int32).max:
                        df[col] = df[col].astype(np.int32)
                    elif mn > np.iinfo(np.int64).min and mx < np.iinfo(np.int64).max:
                        df[col] = df[col].astype(np.int64)    
            # Make float datatypes 32 bit
            else:
                df[col] = df[col].astype(np.float32)
            
            # Print new column type
            print("dtype after: ",df[col].dtype)
            print("******************************")
    # Print final result
    print("___MEMORY USAGE AFTER COMPLETION:___")
    mem_usg = df.memory_usage().sum() / 1024**2 
    print("Memory usage is: ",mem_usg," MB")
    print("This is ",100*mem_usg/start_mem_usg,"% of the initial size")
    return df, NAlist

**Reducing for train data set:**

In [ ]:
train_data, NAlist = reduce_mem_usage(train_data)
print("_________________")
print("")
print("Warning: the following columns have missing values filled with 'df['column_name'].min() -1': ")
print("_________________")
print("")
print(NAlist)

**Reducing for test data set:**

In [ ]:
test_data, NAlist = reduce_mem_usage(test_data)
print("_________________")
print("")
print("Warning: the following columns have missing values filled with 'df['column_name'].min() -1': ")
print("_________________")
print("")
print(NAlist)

**Check again! our RAM. More than 4 GB has got free!**
![ram2](https://1.bp.blogspot.com/-xsHq7cuTlNo/XaaZOmorLlI/AAAAAAAAGTE/6jqT2Mzp6VsQRuUda5Roez7IyM8TRcsMgCLcBGAsYHQ/s1600/ram2.PNG)

In [ ]:
train_data.info()

In [ ]:
test_data.info()

# Add this kernel as Dataset

Now we just save our output as csv files. then you can simply add them to your own kernel.you will save time and memory. But when I start saving this two files I got an error because there is not much space to save both files. **OSError: [Errno 28] No space left on device**

In [ ]:
#train_data.to_csv('train.csv', index=False)
#test_data.to_csv('test.csv', index=False)

# Solution: Save your dataframe as pickle file for fast read.

In [ ]:
train_data.to_pickle("train_data.pkl")
print("train_data size after memory reduction:", os.stat('train_data.pkl').st_size * 1e-6)


test_data.to_pickle("test_data.pkl")
print("test_data size after memory reduction:", os.stat('test_data.pkl').st_size * 1e-6)

**Do not Forget to save your work. This way you can access the downcasted df everytime you load it!**

# How about other ways!


**I have used this great kernel but there are also other ways such as:
**
1. https://www.dataquest.io/blog/pandas-big-data/
2. [optimizing-the-size-of-a-pandas-dataframe-for-low-memory-environment](https://medium.com/@vincentteyssier/optimizing-the-size-of-a-pandas-dataframe-for-low-memory-environment-5f07db3d72e)
3. [pandas-making-dataframe-smaller-faster](https://www.ritchieng.com/pandas-making-dataframe-smaller-faster/)

**The idea behind the this is based on the fact that most of the column only takes few values and pandas can limit those values only to the few categorical value to save the memory.**

**"Often in real-time, data includes the text columns, which are repetitive. Features like gender, country, and codes are always repetitive. These are the examples for categorical data.**

**Categorical variables can take on only a limited, and usually fixed number of possible values. Besides the fixed length, categorical data might have an order but cannot perform numerical operation. Categorical are a Pandas data type.**

**The categorical data type is useful in the following cases −**

**A string variable consisting of only a few different values. Converting such a string variable to a categorical variable will save some memory.**

**The lexical order of a variable is not the same as the logical order (“one”, “two”, “three”). By converting to a categorical and specifying an order on the categories, sorting and min/max will use the logical order instead of the lexical order.**

**As a signal to other python libraries that this column should be treated as a categorical variable (e.g. to use suitable statistical methods or plot types)." (https://www.tutorialspoint.com/python_pandas/python_pandas_categorical_data)**

Thanks to [MJ Bahmani](https://www.kaggle.com/mjbahmani) who created this [kernel](https://www.kaggle.com/mjbahmani/reducing-memory-size-for-ieee)

Pickle code refers to this kaggle [kernal](https://www.kaggle.com/ljjblackpig/3-steps-to-reduce-memory-size-for-the-dataset)